<a href="https://colab.research.google.com/github/rafaelinfopiaui/analise-sentimentos-cpweekend/blob/main/notebooks/Desenvolvimento_Analise_Sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Análise de Sentimentos para a Campus Party Weekend Piauí 2025

**Objetivo:** Desenvolver um protótipo de Machine Learning capaz de classificar o sentimento de textos em português (positivo, negativo, neutro) a partir de dados coletados de redes sociais.

**Projeto de Extensão:** Engenharia de Computação com IA - UNI-CET

**Equipe de Desenvolvedores:**
* Rafael Oliveira
* Ailton Medeiros
* Lais Eulálio
* Antônio Wilker
* Isaac Aragão
* Paula Iranda

**Docente Orientador:** Prof. Dr. Artur Felipe da Silva Veloso

**Etapas deste Notebook:**
1.  **Configuração do Ambiente:** Conexão com o Google Drive e importação de bibliotecas.
2.  **Carga e Análise dos Dados:** Carregar o dataset e fazer uma análise exploratória inicial.
3.  **Pré-Processamento:** Limpeza e preparação dos textos para o modelo.
4.  **Treinamento do Modelo:** Construção, treino e avaliação de um modelo de classificação.
5.  **Salvamento do Modelo:** Exportar o modelo treinado para ser usado na aplicação com Streamlit.

## 1. Configuração do Ambiente

Nesta primeira etapa, a equipe irá conectar o Colab ao Google Drive para acessar os dados, importar todas as bibliotecas que serão utilizadas no projeto e definir os caminhos dos arquivos para manter o código organizado e acessível a todos.

In [ ]:
# 1.1 - Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1.2 - Importar bibliotecas essenciais
import pandas as pd
import numpy as np
import re
import nltk
import joblib

# 1.3 - Baixar pacotes do NLTK (só precisa na primeira vez)
nltk.download('stopwords')
nltk.download('punkt')

# 1.4 - Importar módulos do Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# --- ATENÇÃO: CONFIGURAR CAMINHOS DO PROJETO AQUI ---
# Garanta que o nome da pasta principal seja o mesmo para toda a equipe.
PASTA_PROJETO = '/content/drive/MyDrive/Projeto_CampusParty_Sentimentos'
CAMINHO_DADOS_BRUTOS = f'{PASTA_PROJETO}/data/dataset_bruto.csv'
CAMINHO_DADOS_LIMPOS = f'{PASTA_PROJETO}/data/dados_limpos.csv'
CAMINHO_MODELO = f'{PASTA_PROJETO}/saved_models/modelo_sentimento.joblib'

print("Ambiente configurado com sucesso!")

## 2. Carga e Análise Exploratória dos Dados (EDA)

Nesta etapa, a equipe carrega o dataset e realiza uma verificação rápida para entender sua estrutura, a quantidade de dados, a presença de valores nulos e a distribuição dos sentimentos.

In [ ]:
# 2.1 - Carregar o dataset
try:
    df = pd.read_csv(CAMINHO_DADOS_BRUTOS)
    print("Dataset carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{CAMINHO_DADOS_BRUTOS}'. Verifiquem o caminho e o nome do arquivo no Drive.")

# 2.2 - Visualizar as primeiras linhas
print("Primeiras 5 linhas do dataset:")
display(df.head())

# 2.3 - Verificar informações gerais
print("\nInformações do DataFrame:")
df.info()

# 2.4 - Verificar a distribuição das classes de sentimento
# (A equipe deve substituir 'sentimento' pelo nome real da coluna de classificação)
print("\nDistribuição dos sentimentos:")
print(df['sentimento'].value_counts())

## 3. Pré-processamento e Limpeza dos Textos

Esta é uma das etapas mais críticas. A equipe definiu uma função de limpeza para padronizar o tratamento dos textos de redes sociais, que são inerentemente "sujos" (contêm URLs, menções, etc.). Isso garante que o modelo foque apenas nas palavras que carregam o sentimento.

In [ ]:
# 3.1 - Definir a função de limpeza de texto
def limpar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()
    # Remover URLs
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)
    # Remover menções (@) e hashtags (#)
    texto = re.sub(r'@\w+|#\w+', '', texto)
    # Remover caracteres não-alfabéticos (mantém letras e espaços)
    texto = re.sub(r'[^a-z\s]', '', texto)
    # Tokenização (dividir em palavras)
    tokens = nltk.word_tokenize(texto)
    # Remover stopwords
    stopwords_pt = nltk.corpus.stopwords.words('portuguese')
    tokens_limpos = [palavra for palavra in tokens if palavra not in stopwords_pt]
    # Juntar os tokens de volta em uma string
    return ' '.join(tokens_limpos)

# 3.2 - Aplicar a função de limpeza na coluna de texto
# (A equipe deve substituir 'texto' pelo nome real da coluna de texto)
df['texto_limpo'] = df['texto'].apply(limpar_texto)

# 3.3 - Visualizar o resultado (antes e depois)
print("Comparação do texto original vs. texto limpo:")
display(df[['texto', 'texto_limpo']].head())

# 3.4 - Salvar o DataFrame limpo
df.to_csv(CAMINHO_DADOS_LIMPOS, index=False)
print(f"\nDataFrame limpo salvo em: {CAMINHO_DADOS_LIMPOS}")

## 4. Construção, Treinamento e Avaliação do Modelo

Mãos à obra, equipe! Agora vamos:
1.  Dividir os dados em conjuntos de **treino** e **teste**.
2.  Construir um **`Pipeline`** que automatiza a vetorização do texto (TF-IDF) e a classificação (Regressão Logística).
3.  **Treinar** o modelo com os dados de treino.
4.  **Avaliar** a performance coletivamente com os dados de teste.

In [ ]:
# 4.1 - Definir X (features) e y (target)
X = df['texto_limpo']
y = df['sentimento'] # A equipe deve substituir 'sentimento' se o nome da coluna for diferente

# 4.2 - Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4.3 - Construir o Pipeline do modelo
modelo_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# 4.4 - Treinar o modelo
print("Iniciando o treinamento do modelo...")
modelo_pipeline.fit(X_train, y_train)
print("Treinamento concluído!")

# 4.5 - Fazer previsões no conjunto de teste
y_pred = modelo_pipeline.predict(X_test)

# 4.6 - Avaliar o modelo
acuracia = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo: {acuracia:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

## 5. Salvando o Pipeline Final

Missão cumprida, equipe! O modelo está treinado e avaliado. O último passo neste notebook é salvar o `Pipeline` completo (vetorizador + classificador) em um arquivo `.joblib`. Este arquivo será o "cérebro" da nossa aplicação com Streamlit.

In [ ]:
# 5.1 - Salvar o pipeline treinado
joblib.dump(modelo_pipeline, CAMINHO_MODELO)

print(f"Modelo salvo com sucesso em: {CAMINHO_MODELO}")
print("\nPróximo passo: A equipe irá baixar este arquivo do Google Drive para construir a aplicação com Streamlit!")